In [17]:
# Import necessary libraries
import pandas as pd               # For data manipulation using DataFrames
import numpy as np                # For numerical operations
import matplotlib.pyplot as plt   # For data visualization
import os                         # For operating system-related tasks
import joblib                     # For saving and loading models
import hopsworks                  # For getting access to hopsworks



# Import specific modules from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For data preprocessing
from sklearn.metrics import accuracy_score                        # For evaluating model accuracy

In [18]:
from feature_pipeline import tesla_fg
from feature_pipeline import news_sentiment_fg

In [19]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [20]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549015
Connected. Call `.close()` to terminate connection gracefully.


In [28]:
def create_stocks_feature_view(fs, version):

    #Loading in the feature groups
    tesla_fg = fs.get_feature_group('tsla_stock', version = 1)
    news_sentiment_fg = fs.get_feature_group('news_sentiment', version = 1)

    ds_query = tesla_fg.select(['date','open', 'high', 'low', 'closed', 'volume'])\
        .join(news_sentiment_fg.select_except(['time', 'amp_url', 'image_url']))
    
    transformation_functions = {
        'open': fs.get_transformation_function(name='min_max_scaler'),
        'high': fs.get_transformation_function(name='min_max_scaler'),
        'low' : fs.get_transformation_function(name='min_max_scaler'),
        'closed' : fs.get_transformation_function(name='min_max_scaler'),
        'volume' : fs.get_transformation_function(name='min_max_scaler'),
        'sentiment' : fs.get_transformation_function(name='min_max_scaler'),
    }

    return (fs.create_tesla_feature_view(
        name = 'tsla_stocks_fv',
        query = ds_query,
        labels=['date']
    ), tesla_fg)

In [29]:
try:
    feature_view = fs.get_feature_view("tsla_stocks_fv", version=1)
    tesla_fg = fs.get_feature_group('tsla_stock', version=1)
except:
    feature_view, trans_fg = create_stocks_feature_view(fs, 1)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/549015/featurestores/544840/transformationfunctions). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":270046,"usrMsg":"HOPSFS Connector: HOPSFS","errorMsg":"HopsFs Connector not found"}', error code: 270046, error msg: HopsFs Connector not found, user msg: HOPSFS Connector: HOPSFS